# Een eerste neuraal netwerk

In deze notebook wordt getoond hoe een neuraal netwerk opgebouwd kan worden met Pytorch.
Hierbij wordt er dieper ingegaan op de keuzes van hyperparameters.

In de eerste codecell importeren we de nodige functionaliteiten van pytorch. 
Daarna maken we ook een aantal variabelen aan die we gaan gebruiken voor het aanmaken van het netwerk.
De betekenis van deze parameters is:
* input_size: aantal features in de dataset
* hidden_size: aantal neuronen in de enkele hidden laag
* output_size: aantal neuronen in de outputlaag
* learning_rate: de learning rate van de optimizer
* epochs: aantal keer dat de volledige dataset gebruikt wordt voor training
* batch_size: hoeveel inputs er gegroepeerd worden voordat de gewichten geoptimaliseerd worden
* dropout_rate: het aantal connecties dat op een willekeurige manier zal wegvallen

Voor de input_size heb je meestal geen keuze. Deze wordt bepaald door de gegeven dataset. Elke feature van de input wordt namelijk gematchet op een neuron in de inputlayer.
Ook de output_size wordt bepaald door het probleem. Dit is de gewenste output dus:
* Bij een regressieprobleem is er 1 neuron per ter voorspellen waarde
* Bij classificatie is er 1 neuron per klasse
    * In het geval van 2 klassen kan je gebruik maken van 1 klasse

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

# Configuratie
input_size = 20       # Aantal ingangsfeatures
hidden_size = 64      # Grootte van de verborgen lagen
output_size = 2       # Aantal klassen (voor classificatie)
learning_rate = 0.001 # Learning rate
epochs = 50           # Aantal epochs
batch_size = 32       # Batch grootte
dropout_rate = 0.5    # Dropout rate voor regularisatie

Na het bepalen van de configuratie kan de dataset gemaakt worden. Dit gebeurd met de make_classification functie van scikit-learn om met random data te werken.
In dit voorbeeld is er gekozen voor een classificatieprobleem met 2 klassen.

Deze dataset wordt daarna gesplits in features en labels, en gepslitst in trainings- en testdata.
In pytorch ziet dit eruit als volgt

## Methode 1: Klasieke techniek om NN op te stellen met pytorch
Hierna kan het neuraal netwerk aangemaakt worden.
Eerst gaan we werken met de klassieke en meest manuele manier.
Dit gaat misschien niet de meest beknopte manier zijn maar geeft je wel de meeste mogelijkheden om het model en trainingsproces aan te passen aan je noden.

De klassieke techniek met pytorch is om een klasse te maken die overerft van de Module-klasse.
In deze klasse moet je de volgende zaken aanpassen:

* In de constructor van deze klasse maak je de nodige lagen aan van het neuraal netwerk.
* In de forward functie van de klasse geef je dan hoe deze lagen aan elkaar gekoppeld worden.

In onderstaande voorbeeld worden vier eenvoudige lagen aangemaakt (1 voor de inputs, 2 voor de hiden layers en 1 voor de output layer). De activatiefunctie wordt toegevoegd in de forward functie. In het model hieronder is er gekozen om te werken met een Relu-functie na de input en hidden layer. De output layer bevat een standaard lienaire functie. Dit omdat pytorch beschikt over een speciale loss-functie om een sigmoid/softmax in de output layer overbodig te maken wat efficienter is.

Om dit model te trainen moet er eerst bepaald worden wat er moet geoptimaliseerd worden.
Dit is **de loss-functie** die moet geminimaliseerd worden.
Dit wordt ook bepaald door het op te lossen probleem, namelijk:

* Bij regressie wordt er typisch gekozen voor Mean-Squared-Error (noemt in pytorch MSELoss) of Mean-Squared-Error
* Bij classificatie wordt er gekozen voor
    * BCEWithLogitsLoss bij binaire classificatie (BCE staat voor Binary Cross Entropy )
        * Hierbij moet geen sigmoid activatiefunctie gebruikt worden want dit is reeds ingebouwd in de loss-functie
        * Indien er toch een sigmoid-functie gebruikt wordt in de output-layer: BCELoss
    * CrossEntropy in het geval van multi-class classification (meerdere opties maar slechts 1 klasse te kiezen)
    * BCEWithLogitsLoss bij multilabel classificatie
        *  Aangezien elk outputneuron een binaire classificatie is
     
Daarna wordt de training uitgevoerd door het aantal epoch keer de dataset te gebruiken om de gewichten te optimaliseren.
Hiervoor moet er per epoch het volgende uitgevoerd worden:
* Zet het model in een trainingsmodus
* Geef de data aan het model en doe per batch
    * Bereken de outputs
    * Bereken de error tussen de outputs en gewenste labels
    * Voer backwards propagation uit om gewichten up te daten
* Zet het model in een evaluatie modus
* Bereken de totale loss op de test-date
    *  Dit wordt gebruikt om de prestaties van het model op te volgen en kan gebruikt worden om het trainen eerder te stoppen indien er overfitting gedetecteerd wordt.

De resultaten van het trainingsproces kunnen geplot worden met onderstaande code
De x-as stelt de prestaties van het model voor doorheen het trainingsproces.
Het resulteren model heeft dus de prestaties in de laatste epoch.
Deze figuur kan gebruikt worden om
* De stabiliteit van het model te valideren
    * Een onstabiel model (met veel variatie in de output zal niet goed generaliseren)
* Het aantal gekozen epochs/learning rate te valideren
    * Als de loss-functie nog snel daalt moet het aantal epochs of de learning rate verhoogd worden
* Underfitting detecteren
    * Als de error nog te hoog is voor je doelstelling is ofwel het model niet complex genoeg of de data niet goed genoeg 
* Overfitting detecteren
    * Als er een punt is waarop de test-loss (error op de testdata) terug toeneemt en de train-loss verder daalt. Dan treedt er vanaf die epoch overfitting op
    * Dit kan vermeden worden door meer regularisatie toe te voegen of de complexiteit van het model te verlagen
    * Het aantal epochs reduceren tot dit punt is niet goed want dan verlaag je de prestaties van je model.

## Methode 2: Kortere maar minder flexibele schrijfwijze

Aangezien bovenstaande model een standaard structuur heeft kan het ook herschreven worden als een functioneel model.
In dit geval ziet de code eruit als volgt

Hierbij is de opbouw van het model vereenvoudigd maar **de traningslus moet nog steeds voluit geschreven worden**.

## Methode 3: Keras

Een derde manier om de code te schrijven is door middel van **Keras**. 
Deze manier van code schrijven is identiek aan hoe het eruitziet inden er gebruik gemaakt wordt van **Tensorflow**.
Met keras kan zowel de opbouw van het model als de trainingslus sterk vereenvoudigd worden.

Merk op dat hieronder **wel een softmax nodig is** omdat de standaard crossentropy functies van keras/tensorflow dit vereisen. 
Daarnaast is het ook interessant dat Keras een summary kan uitprinten van een model om een overzicht te krijgen van de architectuur en het aantal parameters dat in elke laag moet getrained worden.

## Oefening 1: Binaire classificatie

Train een feed-forward neurale netwerk voor binaire classificatie met behulp van de make_classification functie van sklearn om een dataset te genereren.

Gebruik een eenvoudig netwerk met één verborgen laag van 64 neuronen en een sigmoid activatiefunctie.
Schrijf je code aan de hand van Keras

**Uitbreiding (ook voor alle andere oefeningen)** Oefen ook de andere code-schrijfwijzen

## Oefening 2: Multiklasse classificatie met Iris Dataset

Train een feed-forward neurale netwerk voor multiklasse classificatie met behulp van de load_iris functie van sklearn om de Iris dataset te laden.
Gebruik een netwerk met drie verborgen laag van 32 neuronen en een softmax activatiefunctie.

Schrijf je code aan de hand van nn.Sequential (tweede schrijfwijze)

## Oefening 3: Regressie met Diabetes Dataset

Train een feed-forward neurale netwerk voor regressie met behulp van de load_diabetes functie van sklearn om de Diabetes dataset te laden.
Gebruik een netwerk met één verborgen laag van 64 neuronen en een gepaste activatiefunctie

Gebruik voor deze oefeningen de klassieke pytorch schrijfwijze

## Oefening 4: Binaire classificatie met Moons Dataset

Train een feed-forward neurale netwerk voor binaire classificatie met behulp van de make_moons functie van sklearn om een maanvormige dataset te genereren.
Pas in deze oefening l2-regularisatie toe ipv dropout.

Gebruik de keras notatie met een pytorch backend (code hiervoor al gegeven hieronder)

## Oefening 5: Multilabel classificatie met Circles Dataset

Train een feed-forward neurale netwerk voor multilabel classificatie met behulp van de make_circles functie van sklearn om een dataset met cirkelvormige klassen te genereren.

Maak gebruik van nn.Module inheritance om dit model te schrijven en zorg voor regularisation door gebruik te maken van de L2-norm.